# Gene essentiality

In [ ]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)

In [ ]:
def get_genes_of_me(me):
    genes = [m.id.split('_')[1] for m in me.metabolites if isinstance(m,cobrame.TranslatedGene)]
    if 'dummy' in genes:
        genes.remove('dummy')
    return list(set(genes))

In [ ]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')
ibsu_directory = join(flat_files.ecoli_files_dir, 'iBsu1103.json') 

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)
ibsu = cobra.io.load_json_model(ibsu_directory)

bsub.optimize()

In [ ]:
met = ibsu.reactions[0]

met.gene_reaction_rule

In [ ]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

## Essentiality

ME-model

In [ ]:
genes = [gene.id for gene in me.genes]

In [ ]:
# essentiality_df_me,flux_responses_me = es.gene_essentiality(me,genes,'me',NP=10,threshold = 0.01,precision=1e-6)
# flux_responses_me.to_csv('essentiality/essentiality_raw_me_results.csv')

In [ ]:
from bacillusme.analysis import sensitivity as ss
flux_responses_me = ss.all_flux_responses(me,genes,NP=10,precision=1.0,max_mu=0.002,
                                       solution=me.solution,
                                       single_change_function='gene_knockout')
flux_responses_me.to_csv('essentiality/essentiality_raw_me_results.csv')